In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. 자신의 주변에 위치한 도서관 파악하기

1-1 사용자의 위치 정보 파악하기

- 사용자의 위치는 GPS나 IP 주소를 기반으로 파악할 수 있습니다.
- Python에서는 geopy 라이브러리를 이용해 위치 정보를 다룰 수 있습니다.
- 예를 들어, 사용자의 IP를 이용해 대략적인 위치를 파악할 수 있습니다.

In [4]:
import requests
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time

# 사용자 위치를 가져오는 함수
def get_user_location(address, retries=3, timeout=10):
    geolocator = Nominatim(user_agent="library_finder")
    for i in range(retries):
        try:
            user_location = geolocator.geocode(address, timeout=timeout)
            if user_location:
                return (user_location.latitude, user_location.longitude)
            else:
                print(f"주소를 지오코딩 할 수 없습니다: {address}")
                return None
        except (GeocoderTimedOut, GeocoderUnavailable) as e:
            print(f"지오코딩 시도 {i+1} 실패: {e}")
            time.sleep(2)  # 재시도하기 전에 대기
    print("최대 재시도 횟수를 초과했습니다. 지오코딩 실패.")
    return None

# 사용자 입력 처리
address = input('주소나 위치 정보를 입력하세요: ')  # 사용자의 주소나 위치 정보
user_location = get_user_location(address)
if user_location:
    latitude, longitude = user_location
    print(f"사용자의 위치: {latitude}, {longitude}")
else:
    print("사용자의 위치 정보를 가져오지 못했습니다.")
    exit()


사용자의 위치: 35.5149524, 129.2938637


1.2 도서관 위치 정보 가져오기

- 지역별 도서관의 위치 데이터를 저장하거나, 공공 API를 이용해 도서관 위치를 받아옵니다. 한국의 경우, 공공데이터 포털에서 도서관 위치와 관련된 데이터를 제공할 수 있습니다.
- geopy.distance 모듈을 사용하여 두 지점 사이의 직선 거리를 계산할 수 있습니다.

In [5]:
import pandas as pd
import math
from geopy.distance import geodesic

# CSV 파일에서 도서관 위치 데이터 읽어오기
def load_libraries(csv_file):
    df = pd.read_csv(csv_file, encoding='UTF8')
    libraries = []
    
    for index, row in df.iterrows():
        if pd.isna(row['LBRRY_LA']) or pd.isna(row['LBRRY_LO']):
            continue
        
        try:
            lat = float(row['LBRRY_LA'])
            lon = float(row['LBRRY_LO'])
            if not math.isfinite(lat) or not math.isfinite(lon):
                continue
        except ValueError:
            continue

        libraries.append({
            'name': row['LBRRY_NM'],
            'coords': (lat, lon)
        })

    return libraries

# CSV 파일 경로 설정 및 도서관 데이터 로드
csv_file = 'C:/개인 프로젝트/boothcamp/도서찾기 프로젝트/도서관 정보_202407.csv'  # CSV 파일 경로
libraries = load_libraries(csv_file)

# 사용자의 위치와 도서관 간의 거리를 계산
def find_nearest_library(user_coords, libraries):
    if user_coords is None:
        return None, None

    nearest_library = None
    shortest_distance = float('inf')
    
    for library in libraries:
        distance = geodesic(user_coords, library['coords']).km
        if distance < shortest_distance:
            shortest_distance = distance
            nearest_library = library
    
    return nearest_library, shortest_distance

# 가장 가까운 도서관 찾기
nearest_library, distance = find_nearest_library(user_location, libraries)

if nearest_library:
    print(f"가장 가까운 도서관은 {nearest_library['name']}이며, 거리는 {distance:.2f} km 입니다.")
else:
    print("주변에 도서관이 없습니다.")


가장 가까운 도서관은 울산남부도서관이며, 거리는 2.32 km 입니다.


1.3 도서관까지의 보행 거리 계산

- 보행 거리나 실제 경로 거리를 계산하려면 Google Maps API나 OpenStreetMap의 osmnx 같은 라이브러리를 사용할 수 있습니다.
- osmnx는 도로 네트워크를 기반으로 경로를 계산하는데 유용합니다.

In [3]:
# OSRM을 이용한 보행 거리 및 시간 계산
def get_walking_distance_osrm(start_coords, end_coords):
    if start_coords is None or end_coords is None:
        return None, None
    
    base_url = 'http://router.project-osrm.org/route/v1/walking/'
    coordinates = f"{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}"
    url = f"{base_url}{coordinates}?overview=false"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['routes']:
            distance = data['routes'][0]['distance'] / 1000  # 미터를 킬로미터로 변환
            duration = data['routes'][0]['duration'] / 60  # 초를 분으로 변환
            
            # 현실적인 소요 시간을 계산
            average_walking_speed_kmph = 5  # 평균 보행 속도 (km/h)
            expected_duration = distance / average_walking_speed_kmph * 60  # 쇼요 시간 (분 단위)
            
            # 선택사항: 필요 시 속도 조정 계수를 적용
            speed_adjustment_factor = 1.2  # 사용자의 걷는 속도에 맞춰 이 계수를 조정
            duration_adjusted = duration * speed_adjustment_factor
            
            # OSRM에서 제공한 시간이 너무 짧으면 현실적인 시간으로 대체
            if duration_adjusted < expected_duration * 0.5:
                duration_adjusted = expected_duration
            
            return distance, duration_adjusted
        else:
            return "경로를 찾을 수 없습니다.", None
    else:
        return "요청 실패.", None

# 보행 거리 및 시간 계산
if nearest_library:
    library_coords = nearest_library['coords']
    walking_distance, walking_duration = get_walking_distance_osrm(user_location, library_coords)
    
    if walking_distance is not None:
        print(f"가장 가까운 도서관까지의 보행 거리는 {walking_distance:.2f} km이며, 소요 시간은 약 {walking_duration:.2f} 분입니다.")
    else:
        print("보행 경로를 찾을 수 없습니다.")


가장 가까운 도서관까지의 보행 거리는 0.73 km이며, 소요 시간은 약 8.81 분입니다.


## 2. 도서관에서 본인이 원하는 책이 있는지 여부 확인

2.1 도서관 책 목록 가져오기

- 각 도서관의 책 목록을 제공하는 API가 있다면 이를 활용해 책 목록을 받아옵니다.
- 예를 들어, 특정 도서관의 웹사이트에서 제공하는 Open API를 사용할 수 있습니다.

In [8]:
import requests
import xml.etree.ElementTree as ET

def check_book_in_library(library_api_url, book_title):
    
    params = {
        #'serviceKey': 'bNwH7fMESNzwAdqKn4rn3mvLVwSHVwt%2Bmze%2Fu1HudJZlAtB0KitYcJHsQn15UdGWTFrSW%2BMmpNd7VxdUTWfLCw%3D%3D',
        'serviceKey': '377f5a2ddbf1d16afa69eba43a18713050e5c3e4ad7f85c05f2182f548df796d',    # 반드시 API 키를 여기에 넣으세요
        'title': book_title
    }
    response = requests.get(library_api_url, params=params)
    
    if response.status_code == 200:
        # XML 데이터를 파싱
        root = ET.fromstring(response.content)
        
        # 예제에 따라 도서의 유무를 판단하는 부분이 다를 수 있습니다.
        # 여기서는 예제로 첫 번째 도서의 상태를 가져오는 방법을 설명합니다.
        for item in root.findall('.//item'):
            book_name = item.find('title').text
            if book_name == book_title:
                available = item.find('loanAvailable').text  # 대출 가능 여부 필드 예시
                return available == 'Y'  # 대출 가능이면 True 반환
        
        return False
    else:
        print("Error: API 요청이 실패했습니다.")
        return False

# 예제 사용
library_api_url = "https://www.nl.go.kr/NL/search/openApi/search.do?key=377f5a2ddbf1d16afa69eba43a18713050e5c3e4ad7f85c05f2182f548df796d&kwd=%ED%86%A0%EC%A7%80"
book_title = ""
is_available = check_book_in_library(library_api_url, book_title)
print(f"Is the book available? {'Yes' if is_available else 'No'}")


In [ ]:
import requests
import xml.etree.ElementTree as ET

# 공공데이터포털에서 받은 개인 API 키
API_KEY = "	377f5a2ddbf1d16afa69eba43a18713050e5c3e4ad7f85c05f2182f548df796d"  # 여기에 개인 API 키를 넣어주세요

# 특정 도서관에서 책의 유무 및 대여 가능 여부를 확인하는 함수
def check_book_availability(lib_code, book_title):
    base_url = "https://www.nl.go.kr/NL/search/openApi/search.do?key=377f5a2ddbf1d16afa69eba43a18713050e5c3e4ad7f85c05f2182f548df796d&kwd=%ED%86%A0%EC%A7%80"
    params = {
        "authKey": API_KEY,
        "libCode": lib_code,  # 도서관 고유 코드
        "keyword": book_title,  # 찾고 싶은 책 제목
        "loaninfoYN": "Y",  # 대출 가능 여부를 포함
        "format": "xml"  # 응답 형식을 XML로 설정
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        
        # API 응답에서 도서 존재 여부 및 대출 가능 여부 확인
        for book in root.findall("doc"):
            book_info = {
                "title": book.findtext("bookname"),
                "author": book.findtext("author"),
                "publisher": book.findtext("publisher"),
                "is_available": book.findtext("loanavailable") == "Y"  # Y이면 대출 가능
            }
            print(f"책 제목: {book_info['title']}")
            print(f"저자: {book_info['author']}")
            print(f"출판사: {book_info['publisher']}")
            print(f"대출 가능 여부: {'가능' if book_info['is_available'] else '불가능'}")
            print("------------------------")
    else:
        print("API 요청 실패:", response.status_code)

# 도서관 코드와 책 제목을 입력
lib_code = "도서관 고유 코드"  # 도서관 고유 코드를 입력하세요
book_title = "원하는 책 제목"  # 원하는 책 제목을 입력하세요

# 책의 대출 가능 여부 확인
check_book_availability(lib_code, book_title)


2.2 원하는 책의 유무 확인

- 사용자가 입력한 책 제목을 검색해 각 도서관의 API에서 해당 책이 있는지 확인합니다. 여러 도서관에 대해 반복해서 검색할 수 있습니다.

In [ ]:
libraries = [
    {"name": "Library 1", "api_url": "http://example.com/library1_api"},
    {"name": "Library 2", "api_url": "http://example.com/library2_api"},
    # more libraries...
]

for library in libraries:
    is_available = check_book_in_library(library['api_url'], book_title)
    if is_available:
        print(f"{book_title} is available at {library['name']}")
    else:
        print(f"{book_title} is not available at {library['name']}")


최종적으로 애플리케이션을 웹 애플리케이션으로 확장하려면
- 웹 프레임워크: Flask나 Django를 사용하여 API와 사용자 인터페이스를 구축할 수 있습니다.
- 지도 API: Google Maps, OpenStreetMap 등을 이용해 도서관 위치를 지도에 표시하고, 경로를 안내할 수 있습니다.
-  프론트엔드: HTML, CSS, JavaScript로 사용자 인터페이스를 구성합니다.

추가 고려 사항
- 데이터베이스: 도서관 위치와 책 목록을 데이터베이스에 저장하고 관리할 수 있습니다. SQLite, PostgreSQL, MySQL 등을 사용할 수 있습니다.
- 사용자 인증: 사용자가 개인 계정을 통해 특정 도서관을 즐겨찾기하거나 책을 예약할 수 있도록 하는 기능을 추가할 수 있습니다.

위의 구조를 기반으로 프로젝트를 시작하면, 단계별로 구현해나갈 수 있을 것입니다. 처음에는 간단한 콘솔 애플리케이션으로 시작한 뒤, 점차 웹 애플리케이션으로 확장해 나가면 됩니다.